#Features to be extracted:
    issuer_name
 * security_description
 * cusip_id
 * isin_id
 * original_face_amount
 * maturity_date
 * coupon_type_code
 * cpn_pay_frq_code
 * is_callable
 
 #goal of the analysis:
 extract the above features and use NLP to extract same features in other documents

In [77]:
import pandas as pd
from bs4 import BeautifulSoup as BS
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from collections import Counter
import nltk
import spacy
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()
import re
from spacy.matcher import Matcher

In [2]:
#current working direcgtory
cwd = os.getcwd()

In [167]:
#To read one single html file saved on local laptop. Note- exact filepath is important

soup = BS(open("/Volumes/NSS/Alliance_bernstein/dq9-edgar-the-razorbacks/data/424B2_a19-18449_4424b2.htm"), "html.parser")
text_a19 = soup.get_text()
#print(text_a19)

#remove scripts and style from html document
soup = BeautifulSoup(html_src, "lxml")
[x.extract() for x in soup.findAll(['script', 'style'])]

In [4]:
import os
filepaths = [os.path.join('data', file) for file in os.listdir('data/') if '.htm' in file[-4:]]
files = []
for path in filepaths:
    with open(path, 'rb') as file:
        files.append(file.read())

In [166]:

html = files[1]
soup = BS(html)

# kill all script and style elements
#for script in soup(["script", "style"]):
    #script.extract()    # rip it out

# get text
text1 = soup.get_text()

# break into lines and remove leading and trailing space on each
#lines = (line.strip() for line in text1.splitlines())
# break multi-headlines into a line each
#chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
#text1 = '\n'.join(chunk for chunk in chunks if chunk)

#print(text1)
#text = soup.get_text()
#print(text)

In [168]:
#Import the 6th element in the list of files files[6]
html6 = files[6]
soup6 = BS(html6)

# kill all script and style elements
#for script in soup(["script", "style"]):
    #script.extract()    # rip it out

# get text
text6 = soup6.get_text()

# break into lines and remove leading and trailing space on each
#lines = (line.strip() for line in text1.splitlines())
# break multi-headlines into a line each
#chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
#text1 = '\n'.join(chunk for chunk in chunks if chunk)

In [6]:
#use doc similarity to check which documents are similar

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/suneethi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
tokens_a19=nltk.word_tokenize(text_a19)

In [9]:
len(tokens_a19)

1013

In [165]:
#tokens_a19

In [11]:
tokens_a19[:2]

['PROSPECTUS', 'and']

In [12]:
tokens1=nltk.word_tokenize(text1)
len(tokens1)

948

In [164]:
#tokens1

In [14]:
tokens1[:2]

['PROSPECTUS', 'and']

In [15]:
from spacy.lang.en import English

In [16]:
nlp=English()

In [17]:
nlp = spacy.load('en')
nlp.entity

In [169]:
doc=nlp(text_a19)
doc1=nlp(text1)
doc6=nlp(text6)

In [97]:
#doc.ents
#print(doc.ents)

In [163]:
#doc1.ents

In [57]:
#for ent in doc.ents:
    #print(ent.label_, ent.text)

for ent in doc1.ents:
    print(ent.label_, ent.text)

In [108]:
labels = [x.label_ for x in doc1.ents]
#Counter(labels)

In [109]:
items = [x.text for x in doc.ents]
#Counter(items).most_common(30)

In [170]:
sentences = [x for x in doc.sents]
sentences1= [x for x in doc1.sents]
sentences6=[x for x in doc6.sents]

#convert the text into sentences and words - did not get what i wanted
sentences_token = sent_tokenize(text1)
unique_tokens = set(word_tokenize(text1))
unique_tokens

match = re.search("CUSIP", text1)
print(re.match(match, text1))

In [147]:
#This will only give the word CUSIP. what we need is the CUSIP number. User Regex
pattern1 = [{'UPPER': 'r"[A-Z]"'}, {'TEXT':'/'}, {'UPPER': 'r"[A-Z]"'}, {'POS': 'PROPN'}]
matcher = Matcher(nlp.vocab)
matcher.add('CUSIP /', None, pattern1)

matches1=matcher(doc1)

In [148]:
matches1

[(11353242767468343225, 163, 167)]

In [161]:
type (matches1)

list

In [149]:
for match_id, start, end in matcher(doc1):
    span = doc1[start:end]
    print('Matched span:', span.text)
# Get the span's root token and root head token
    print('Root token:', span.root.text)
    print('Root head token:', span.root.head.text)

Matched span: 24422EVA4 /   US24422EVA45
Root token: US24422EVA45
Root head token: ISIN


In [162]:
type(span.text)

str

In [150]:
pattern1 = [{'UPPER': 'r"[A-Z]"'}, {'TEXT':'/'}, {'POS': 'PROPN'}]
matcher = Matcher(nlp.vocab)
matcher.add('CUSIP /', None, pattern1)



In [151]:
matches=matcher(doc)
for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print('Matched span:', span.text)
# Get the span's root token and root head token
    print('Root token:', span.root.text)
    print('Root head token:', span.root.head.text)

Matched span: CUSIP / ISIN
Root token: ISIN
Root head token: ISIN
Matched span: 24422EUZ0 / US24422EUZ05
Root token: 24422EUZ0
Root head token: Date


In [200]:
pattern6 = [{'UPPER': 'r"[A-Z]"'}, {'TEXT':'/'}, {'UPPER': 'r"[A-Z]"'}]
matcher = Matcher(nlp.vocab)
matcher.add('CUSIP / ISIN', None, pattern1)

matches6=matcher(doc6)
for match_id, start, end in matcher(doc6):
    span = doc[start:end]
    print('Matched span:', span.text)

Matched span: all such persons
Matched span: persons together being


In [199]:
matches6

[(8638425075047199085, 817, 820), (8638425075047199085, 819, 822)]

In [175]:
#set(text1)
#sorted(w for w in set(text1) if  w.isupper())
#[w.upper() for w in text1]
for token in doc1:
     if token.isupper():
         print(token, 'is a uppercase word')

AttributeError: 'spacy.tokens.token.Token' object has no attribute 'isupper'

In [132]:
displacy.render(nlp(str(sentences1[:30])), style='dep', jupyter = True, options = {'distance': 100})

In [143]:
displacy.render(nlp(str(sentences[:30])), style='dep', jupyter = True, options = {'distance': 100})

In [171]:
displacy.render(nlp(str(sentences6[:30])), style='dep', jupyter = True, options = {'distance': 100})

In [25]:
dict([(str(x), x.label_) for x in nlp(str(sentences[30])).ents])

{'LLC': 'ORG', '10,000,000': 'CARDINAL'}

In [42]:
dict([(str(x), x.label_) for x in nlp(str(sentences1[30])).ents])

{'the Securities Exchange Act': 'LAW',
 '1934': 'DATE',
 'two business days': 'DATE'}

In [55]:
#sentence_endings = r"[.?!]"

#capitalized_words = r"[A-Z]\w+"
#print(re.findall(capitalized_words, text1))

In [26]:
# extract the issue date by looking for "issue" or its other forms and put in a table

In [27]:
# Extracting CUSIP/ISIN:
#function:
   # if token is CUSIP/ISIN , then separate using / and take the first number
   # if token is CUSIP, then extract the corresponding number separated by space
#CUSIP - PROPN
#ISIN - NOUN or PROPN

In [ ]:
for ent in txt.entities:
    # Check whether the entity contains 'Márquez' or 'Gabo'
    if 'Márquez' in ent or "Gabo" in ent:
        # Increment count
        count = count+1

# Print count
print(count)


In [28]:
#remove spaces in the html page

In [29]:
token=doc[4]
print(token.text)

PRICING


In [50]:
for ent in doc.ents:
    print(ent.text, ent.label_)

34 CARDINAL
April 7, 2017 DATE
1 CARDINAL
June DATE
27 CARDINAL
Filed Pursuant WORK_OF_ART
2 CARDINAL
JOHN DEERE ORG
Date of Issue ORG
13 CARDINAL
2022 CARDINAL
The Medium-Term Notes WORK_OF_ART
U.S. GPE
September DATE
12 CARDINAL
June DATE
13 CARDINAL
$400,000,000  


  

  

   


 MONEY
Reuters Page LIBOR01 ORG
49 CARDINAL
Actual/360 ORG
Adjusted  


  

  

   


Interest Reset Dates ORG
Quarterly DATE
the 13th of March DATE
September DATE
December DATE
September DATE
Two CARDINAL
London GPE
Interest Reset Date  


  

  

   


Interest Payment Dates ORG
Quarterly DATE
the 13th of March DATE
September DATE
December DATE
December DATE
2019 DATE
0.000% PERCENT
Modified Following PERSON
100.000% PERCENT
September DATE
12 CARDINAL
Plan of Distribution WORK_OF_ART
Citigroup Global   Markets Inc. ORG
90,000,000 MONEY
Deutsche Bank   Securities Inc. ORG
90,000,000 CARDINAL
HSBC Securities   (USA) Inc. ORG
90,000,000 CARDINAL
J.P. Morgan    PERSON
90,000,000 CARDINAL
BNP Paribas   Securit

# to extract texts from a table:
results = []
filepaths = [os.path.join('data', file) for file in os.listdir('data/') if '.htm' in file[-4:]]
for i in filepaths:
    with open(i, 'rb') as file:
        html_text = BS(file.read())
        table_html = str(html_text)
        r = pd.read_html(table_html)
        for g in r:
            tables = g.dropna(how='all')
            tables = tables.dropna(axis=1, how='all')
            tables = tables.dropna()
            tables = tables.T
            results.append(tables)
results2 = [t for t in results if len(t) > 0] #removing tables with 0 rows
final = [] 
for g in results2:
    new_header = g.iloc[0] #grab the first row for the header
    g = g[1:] #take the data less the header row
    g.columns = new_header #set the header row as the df header
    final.append(g)

In [ ]:
final[2]

In [ ]:
# to parse through hundreds of html files 
#source: https://rushter.com/blog/python-fast-html-parser/
from time import time

import warc
from bs4 import BeautifulSoup
from selectolax.parser import HTMLParser


def get_text_bs(html):
    tree = BeautifulSoup(html, 'lxml')

    body = tree.body
    if body is None:
        return None

    for tag in body.select('script'):
        tag.decompose()
    for tag in body.select('style'):
        tag.decompose()

    text = body.get_text(separator='\n')
    return text


def get_text_selectolax(html):
    tree = HTMLParser(html)

    if tree.body is None:
        return None

    for tag in tree.css('script'):
        tag.decompose()
    for tag in tree.css('style'):
        tag.decompose()

    text = tree.body.text(separator='\n')
    return text


def read_doc(record, parser=get_text_selectolax):
    url = record.url
    text = None

    if url:
        payload = record.payload.read()
        header, html = payload.split(b'\r\n\r\n', maxsplit=1)
        html = html.strip()

        if len(html) > 0:
            text = parser(html)

    return url, text


def process_warc(file_name, parser, limit=10000):
    warc_file = warc.open(file_name, 'rb')
    t0 = time()
    n_documents = 0
    for i, record in enumerate(warc_file):
        url, doc = read_doc(record, parser)

        if not doc or not url:
            continue

        n_documents += 1

        if i > limit:
            break

    warc_file.close()
    print('Parser: %s' % parser.__name__)
    print('Parsing took %s seconds and produced %s documents\n' % (time() - t0, n_documents))